In [59]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pickle
from xgboost import XGBClassifier



def plot_confusion_matrix(y_true, y_pred, label,
                          cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data

    print(cm)

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=label, yticklabels=label,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


import pandas as pd

In [93]:

features = ['Date', 'Payrolls_3mo_vs_12mo', 'Effective_Fed_Funds_12_chg', 'CPI_All_Items_3_mo_annualised',
            '10Y_Treasury_Rate_12_chg', '3M_10Y_Treasury_Spread', 'S&P_500_Index_12_chg','Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/Data/final_features.csv')[::-1]
predicted_probability=pd.DataFrame()
past_predict = pd.DataFrame()
test_prob = pd.DataFrame()
recession_data = df[features]
#df_features.join(pd.get_dummies(df['Recession']))
output_feature =['Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']


In [95]:
x_training_data=recession_data[recession_data['Date'] <= '2018-04-01']
x_testing_data = recession_data[recession_data['Date'] >= '2018-04-01']


In [97]:
X_train, X_test, y_train, y_test= train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.20)

,Recession_in_6mo,Recession_in_12mo,Recession_in_24mo
616,0,1,0
394,0,0,0
298,0,0,0
179,0,0,0
775,0,0,1
...,...,...,...
642,0,0,0
474,0,1,1
477,0,1,1
277,0,0,0


In [102]:
from imblearn.over_sampling import ADASYN
adasyn = ADASYN(random_state=88)
X_adasyn, y_adasyn = adasyn.fit_resample(X_train.iloc[:,1:7], y_train['Recession_in_24mo'])

In [124]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
models = [{'name': 'lg','label': 'Logistic Regression',
           'classifier': LogisticRegression(random_state=88),
           'grid': {"C":np.logspace(-3,3,7), "penalty":["l2"]}},
          
          {'name': 'knn','label':'K Nearest Neighbors',
           'classifier':KNeighborsClassifier(),
           'grid': {"n_neighbors":np.arange(8)+1}},
          {
              'name':'xg','label':'xgboost','classifier':XGBClassifier(objective ='binary:logistic', learning_rate = 0.1,
                 booster='gbtree',alpha = 10, n_estimators = 10),'grid':{
                  'max_depth':range(3,10,1),
                'gamma':[i/10.0 for i in range(0,5)],
                 'colsample_bytree':[i/10.0 for i in range(1,10)],
                 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05] 
              }
          },
          {'name': 'sv', 'label': 'SVC (RBF)',
           'classifier':SVC(probability=True,
                        tol=1e-3, random_state=123,
                        class_weight='balanced'),
           'grid': {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}}]

In [125]:
from sklearn.metrics import roc_auc_score
def model_selection(classifier, name, grid, X_train, y_train, scoring,i):
    
    gridsearch_cv=GridSearchCV(classifier, 
                               grid,
                               cv=5, 
                               scoring = scoring,verbose=2,n_jobs=2)
    
    gridsearch_cv.fit(X_train, y_train)
    
    results_dict = {}
    
    results_dict['classifier_name'] = name    
    results_dict['classifier'] = gridsearch_cv.best_estimator_
    results_dict['best_params'] = gridsearch_cv.best_params_
    results_dict['ROC_AUC'] = gridsearch_cv.best_score_
    filename = i+'.'+name
    pickle.dump(gridsearch_cv.best_estimator_, open(filename, 'wb'))
    
    return(results_dict)


In [126]:
for i in output_feature:
    results = []
    X_adasyn, y_adasyn = adasyn.fit_resample(X_train.iloc[:,1:7], y_train[i])
    for m in models:  
        print(m['name'])    
        results.append(model_selection(m['classifier'], 
                                   m['name'],
                                   m['grid'],
                                   X_adasyn, 
                                   y_adasyn, 
                                   'roc_auc',i))      
        print('completed')
        results_df = pd.DataFrame(results).sort_values(by='ROC_AUC', ascending = False)
        results_df.to_csv(i+'.csv',index=False)
    
    
    








lg
Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  35 out of  35 | elapsed:    2.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


completed
knn
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    0.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


completed
xg
Fitting 5 folds for each of 1575 candidates, totalling 7875 fits


...

completed
xg
Fitting 5 folds for each of 1575 candidates, totalling 7875 fits


[Parallel(n_jobs=2)]: Done 862 tasks      | elapsed:    5.1s
[Parallel(n_jobs=2)]: Done 4008 tasks      | elapsed:   24.8s
[Parallel(n_jobs=2)]: Done 7875 out of 7875 | elapsed:   55.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


completed
sv
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    2.8s
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    2.9s finished


completed


In [138]:
roc_test=[]

df = pd.read_csv("/Users/sebin/Desktop/companyandngo/src/Models/Recession_in_24mo.csv")
classifier = df['classifier'].to_list() 

for i in classifier:
    X_adasyn, y_adasyn = adasyn.fit_resample(X_train.iloc[:,1:7], y_train['Recession_in_6mo'])
    s=eval(i).fit(X_adasyn,y_adasyn)
    pp=s.predict_proba(X_test.iloc[:,1:7])
    score = roc_auc_score(y_test, pp)
    roc_test.append(score)

    

In [140]:
df = pd.read_csv("/Users/sebin/Desktop/companyandngo/src/Models/Recession_in_24mo.csv")

In [142]:
df.columns

Index(['classifier_name', 'classifier', 'best_params', 'ROC_AUC',
       'test_ROC_AUC'],
      dtype='object')

In [28]:
l = results[0].get('classifier').predict_proba(X_test.iloc[:,1:7])[:,1]

In [31]:
from sklearn.metrics import roc_auc_score

In [ ]:
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(testy, ns_probs)

In [ ]:

param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf'],'probability':[True],'class_weight':['balanced'],'tol':[1e-3]}

In [79]:
import xgboost as xb
X_train['Recession_in_6mo'].value_counts()


0    514
1     89
Name: Recession_in_6mo, dtype: int64

In [130]:


eval(classifier[0])

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [119]:
roc_test

[0.40352114426199837,
 0.3782616908436191,
 0.4293197848926318,
 0.430237270192837,
 0.3932180272971814]

In [52]:
type(classifier)

list

In [ ]:
from sklearn.model_selection import GridSearchCV  
for i in output_feature:
    grid = GridSearchCV(svm.SVC(),param_grid,verbose=2,scoring='recall',cv=10)
    grid.fit(X_train.iloc[:,1:7],y_train[i])
    svmc=grid.best_estimator_
    predicted_probability[i+"_probability"] = svmc.predict_proba(x_testing_data.iloc[:,1:7])[:,1]
    past_predict[i+"_probability"]=svmc.predict_proba(X_train.iloc[:,1:7])[:,1]
    test_prob[i+"_probability"]=svmc.predict_proba(X_test.iloc[:,1:7])[:,1]
    filename = i+'.sv'
    pickle.dump(svc_model, open(filename, 'wb'))
      
    



In [ ]:
for i in output_feature:
    svm_ac = {}
    for x, y in [(x, y) for x in c for y in gamma]:
        k = x, y
        # initalize the'model' and passing the SVM Classifier with respect to C and gamma value
        model = svm.SVC(C=x, kernel='rbf', gamma=y, probability=True,
                        tol=1e-3,
                        class_weight='balanced')
        accuracy = cross_val_score(model, X_train.iloc[:,1:7], y_train[i], cv=10, scoring='recall')
        svm_ac[k] = np.mean(accuracy)
        print("\n{:20} {:25}".format('Parameter', 'Average accuracy'))
        print('{}\t{:25}'.format(k, np.mean(accuracy)))
    print(f"\n----{i}------")
    acc_score = max(svm_ac, key=svm_ac.get)
    print(f"The best tuning parameter{acc_score}")
    svc_model = svm.SVC(C=acc_score[0], kernel='rbf', gamma=acc_score[1], probability=True,
                        tol=1e-3, random_state=123,
                        class_weight='balanced')
   
    # fitting the model with training data
    svc_model.fit(X_train.iloc[:,1:7], y_train[i])
    # predicting the model with the test data
    y_predict = svc_model.predict(X_test.iloc[:,1:7])

    # evaluating the accuracy with the ground truth and predicted value
    model_acc = accuracy_score(y_test[i], y_predict)

    print("Model Accuracy is: {}".format(model_acc))
    # classification_report((y_true, y_pred)
    predicted_probability[i+"_probability"] = svc_model.predict_proba(x_testing_data.iloc[:,1:7])[:,1]
    past_predict[i+"_probability"]=svc_model.predict_proba(X_train.iloc[:,1:7])[:,1]
    test_prob[i+"_probability"]=svc_model.predict_proba(X_test.iloc[:,1:7])[:,1]
    filename = i+'.sv'
    pickle.dump(svc_model, open(filename, 'wb'))
      

In [ ]:

predicted_probability['Date']= x_testing_data['Date'].tolist()

In [ ]:
predicted_probability

In [ ]:
past_predict['Date']= X_train['Date'].tolist()
test_prob['Date']=X_test['Date'].tolist()


In [ ]:
past_predict.append(test_prob)

In [ ]:
past_predict.append(predicted_probability)

In [ ]:
file=['Recession_in_6mo.xg','Recession_in_12mo.xg']

for i in file:
    svmc = pickle.load(open(i, 'rb'))
    print(svmc.predict_proba(x[:,1:7])[:,1])
    
    
    



In [ ]:
test=recession_data[recession_data['Date'] == '2007-02-01']

test

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=2000, random_state=0,class_weight='balanced')
clf.fit(X_train.iloc[:,1:7], y_train)

In [ ]:
y_predict = clf.predict_proba(X_test.iloc[:,1:7])

  

In [ ]:
y_predict

In [ ]:
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/website/house/sydney_new_data.csv')

In [ ]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df.iloc[1:1000,:]

In [ ]:
df.to_csv('/Users/sebin/Desktop/companyandngo/website/melboure_house.csv')

In [ ]:
df.columns

In [ ]:
##logistic regression

In [ ]:
df.iloc[1:1000,:].to_csv('/Users/sebin/Desktop/companyandngo/website/1000_sydney.csv')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
c = [1, 10, 100, 1000]
gamma = [1e-3, 1e-4,.04]
features = ['Date', 'Payrolls_3mo_vs_12mo', 'Effective_Fed_Funds_12_chg', 'CPI_All_Items_3_mo_annualised',
            '10Y_Treasury_Rate_12_chg', '3M_10Y_Treasury_Spread', 'S&P_500_Index_12_chg','Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/Data/final_features.csv')[::-1]
predicted_probability=pd.DataFrame()
past_predict = pd.DataFrame()
test_prob = pd.DataFrame()
recession_data = df[features]
#df_features.join(pd.get_dummies(df['Recession']))
output_feature =['Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']



In [ ]:
x_training_data=recession_data[recession_data['Date'] <= '2018-02-01']
x_testing_data = recession_data[recession_data['Date'] >= '2018-02-01']
X_train, X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [ ]:

# implementing a Logistic Regression model with random hyperparameters
LR = LogisticRegression(penalty='l1',dual=False,max_iter=110, solver='liblinear')
LR.fit(X_train,Y_train)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=110, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


# In[158]:


#setting the parameters
dual=[True,False]
max_iter=[100,110,120,130,140]
C = [1.0,1.5,2.0,2.5,3.0,3.5,4.0]
param_grid = dict(dual=dual,max_iter=max_iter,C=C)


# In[159]:


#implementing a Grid search on our defined hyperparameters
LR = LogisticRegression(penalty='l2')
G_search = GridSearchCV(estimator=LR, param_grid=param_grid, cv = 3, n_jobs=-1)
import time

starting_time = time.time()
G_search_results = G_search.fit(X_train,Y_train)
# Summarizing the  results
print("Best accuracy: %f gained by using this set of parameters%s" % (G_search_results.best_score_, G_search_results.best_params_))
print("time of execution: " + str((time.time() - starting_time)) + ' ms')


# In[160]:


#implementing a Random search on our defined hyperparameters
R_search = RandomizedSearchCV(estimator=LR, param_distributions=param_grid, cv = 3, n_jobs=-1)
starting_time = time.time()
R_search_results = R_search.fit(X_train,Y_train)
# Summarizing the  results
print("Best accuracy: %f gained by using this set of parameters%s" % (R_search_results.best_score_, R_search_results.best_params_))
print("time of execution: " + str((time.time() - starting_time)) + ' ms')
